In [1]:
import pandas as pd
import numpy as np

from Code.database import engine

from itertools import combinations

import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_white'

# Tables

In [2]:
def get_table_from_list(l, name):
    table = (
        pd.Series(
            l, 
            name=name, 
        )
        .to_frame()

        .assign(Order=lambda df: df[name].expanding().count())
    )
    return(table)

## Body

## Nutrition

### Meals

In [3]:
meals = ['Breakfast', 'Lunch', 'Snack', 'Dinner', 'Pre-workout', 'Intra-workout', 'Post-workout']

df_meals = get_table_from_list(meals, 'Meal')
display(df_meals)

,Meal,Order
0,Breakfast,1.0
1,Lunch,2.0
2,Snack,3.0
3,Dinner,4.0
4,Pre-workout,5.0
5,Intra-workout,6.0
6,Post-workout,7.0


### Aliments

## Workout

### Exercises

In [4]:
exercises = {
    'Vertical push': {'Bw': ['Pike push-ups', 'Elevated pike push-ups']}, 
    'Horizontal push': {'Bw': ['Push-ups', 'Deep push-ups']}, 
    'Elbow extension': {'Bw': ['Bench dips', 'Dips']}, 

    'Vertical pull': {'Bw': ['Pull-ups']}, 
    'Horizontal pull': {'Bw': ['Australian pull-ups']}, 
    'Elbow flexion': {'Bw': ['Chin-ups']}, 

    'Knee dominant': {'Bw': ['Squats', 'Wall-assisted pistol-squats']}, 
    'Hip dominant 1': {'Bw': ['Romanian deadlifts', 'Wall-assisted single-leg romanian deadlifts']}, 
    'Hip dominant 2': {'Bw': ['Hip-thrusts', 'Deep hip-thrusts', 'Single-leg deep hip-thrusts']}, 

    'Triceps':  {'Bw': ['Tricep extensions (very high bar)', 'Tricep extensions (high bar)', 'Tricep extensions (medium bar)']}, 
    'Biceps': {'Bw': ['Declined bicep curls']}, 

    'Quads':  {'Bw': ['Natural leg extensions']}, 
    'Hamstrings':  {'Bw': ['Nordic hip hinges', 'Nordic curls']}, 

    'Upper abs': {'Bw': ['Kneeling ab rollouts']}, 
    'Lower abs': {'Bw': ['Hanging knee raises']}, 
    'Obliques': {'Bw': ['']}, 

    'Support': {'Bw': ['Support hold']}, 
    'Hang': {'Bw': ['Hang hold']}, 
}

exercises = ([
    (pattern, resistance, progression, level+1) 
    for pattern, resistances in exercises.items()
    for resistance, progressions in resistances.items()
    for level, progression in enumerate(progressions)
])
df_exercises = (
    pd.DataFrame(
        exercises, 
        columns=['Pattern', 'Resistance', 'Progression', 'Level'], 
    )
    
    .assign(Order=lambda df: df['Progression'].expanding().count())
)
display(df_exercises)

,Pattern,Resistance,Progression,Level,Order
0,Vertical push,Bw,Pike push-ups,1,1.0
1,Vertical push,Bw,Elevated pike push-ups,2,2.0
2,Horizontal push,Bw,Push-ups,1,3.0
3,Horizontal push,Bw,Deep push-ups,2,4.0
4,Elbow extension,Bw,Bench dips,1,5.0
5,Elbow extension,Bw,Dips,2,6.0
6,Vertical pull,Bw,Pull-ups,1,7.0
7,Horizontal pull,Bw,Australian pull-ups,1,8.0
8,Elbow flexion,Bw,Chin-ups,1,9.0
9,Knee dominant,Bw,Squats,1,10.0


## *Initialize tables*

In [5]:
df_meals.rename_axis('Id').to_sql('Meals', engine, if_exists='append')

df_exercises.rename_axis('Id').to_sql('Exercises', engine, if_exists='append')

# Weights calculation

## Bands weight

In [6]:
bands = [5, 15, 25, 35, 45, 60]

(
    pd.DataFrame([
        {
            'Weight': np.sum(combination), 
            'Nb_bands': nb_bands+1, 
            'Bands': combination, 
        }
        for nb_bands in range(len(bands))
        for combination in combinations(bands, nb_bands+1)
    ])
    
    .sort_values(['Weight', 'Nb_bands'])
    
    .query('Weight >= 0')
)

,Weight,Nb_bands,Bands
0,5,1,"(5,)"
1,15,1,"(15,)"
6,20,2,"(5, 15)"
2,25,1,"(25,)"
7,30,2,"(5, 25)"
...,...,...,...
59,160,5,"(5, 15, 35, 45, 60)"
55,165,4,"(25, 35, 45, 60)"
60,170,5,"(5, 25, 35, 45, 60)"
61,180,5,"(15, 25, 35, 45, 60)"


## HandGripper weight

In [7]:
weight_min, weight_max = 10, 40
nb_turns = 18

(
    pd.DataFrame([
        {
            'Nb_turns': k, 
            'Weight': weight_min + k*(weight_max-weight_min)/nb_turns, 
        }
        for k in range(nb_turns+1)
    ])
    
    .query('Weight > 0')
)

,Nb_turns,Weight
0,0,10.000000
1,1,11.666667
2,2,13.333333
3,3,15.000000
4,4,16.666667
5,5,18.333333
6,6,20.000000
7,7,21.666667
8,8,23.333333
9,9,25.000000


# *Drafts*

In [8]:
df_exercises.columns

Index(['Pattern', 'Resistance', 'Progression', 'Level', 'Order'], dtype='object')

In [9]:
(
    px.sunburst(
        df_exercises, 

        path=['Pattern', 'Progression'], 
        values='Level', 

        color='Pattern', 
    )
)

In [5]:
df_ciqual = (
    pd.read_excel('Data/Table Ciqual 2020_FR_2020 07 07.xls')
    [[
        'alim_code', 'alim_nom_fr', 
        
        'Energie, N x facteur Jones, avec fibres  (kcal/100 g)', 
        'Protéines, N x facteur de Jones (g/100 g)', 
        'Glucides (g/100 g)', 
        'Lipides (g/100 g)', 
        'Alcool (g/100 g)', 
        #'Fibres alimentaires (g/100 g)', 
    ]]
    .assign(
        alim_code=lambda df: df.alim_code.astype(str), 
    )
    .set_index('alim_code')
    
    .mask(lambda df: df=='-')
    .mask(lambda df: df=='traces', 0)
    
    .replace('< ', '', regex=True)
    .replace(',', '.', regex=True)
    
    .set_index('alim_nom_fr', append=True)
    .astype('float', errors='ignore')
    .reset_index('alim_nom_fr')

    .assign(**{
        'Energie, N x facteur Jones, avec fibres  (kcal/100 g)': lambda df: df['Energie, N x facteur Jones, avec fibres  (kcal/100 g)'].fillna(4*df['Protéines, N x facteur de Jones (g/100 g)']+4*df['Glucides (g/100 g)']+9*df['Lipides (g/100 g)']+7*df['Alcool (g/100 g)']), 
    })
)
df_ciqual

,alim_nom_fr,"Energie, N x facteur Jones, avec fibres (kcal/100 g)","Protéines, N x facteur de Jones (g/100 g)",Glucides (g/100 g),Lipides (g/100 g),Alcool (g/100 g)
alim_code,,,,,,
24999,Dessert (aliment moyen),281.587,4.63,36.60,12.90,0.081
25601,Salade de thon et légumes. appertisée,109.860,9.15,7.74,4.70,0.000
25602,Salade composée avec viande ou poisson. appert...,105.540,8.06,6.40,5.30,0.000
25605,Champignons à la grecque. appertisés,56.070,2.08,3.95,3.55,0.000
25606,Salade de pommes de terre. fait maison,124.120,2.68,9.90,8.20,0.000
...,...,...,...,...,...,...
13166,Dessert lacté infantile nature sucré ou aux fr...,100.000,3.19,12.50,3.50,0.000
13167,Céréales instantanées. poudre à reconstituer. ...,393.000,5.10,88.30,1.70,0.000
13168,Céréales instantanées. poudre à reconstituer. ...,391.000,10.00,81.10,2.00,0.000
